In [2]:
import os
from Bio import SeqIO
import io

In [6]:
blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/2.Blast/1.Blast_output"
species = "Pararge_aegeria"


In [7]:

# def mafft_alignement_and_processing(coordinate_file_list, current_exon_number):
    

In [8]:
query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/2.Blast/0.Query"

list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
coordinate_file_name = ''
for file_names in list_of_files_in_species_folder:
    if file_names.endswith("_coordinates.csv"):
        coordinate_file_name = file_names
if coordinate_file_name =='':
    print(f"Coordinate file error")
    assert False

with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
    coordinate_file_lines = temp_file_open.readlines()
print(coordinate_file_lines[0])
for i in range(1,len(coordinate_file_lines)):
    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
    current_exon_coordinates = coordinate_file_lines[i].split(",")
    if coordinate_file_lines_split[5] == "Y":
        if i == 1:
            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
            assert False
        previous_exon_number = i-1
        next_exon_number = i+1
        previous_exon_coordinates = ''
        next_exon_coordinates = ''
        while True:
            if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                break
            else:
                proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                if proceed_prompt.lower()[0] == "y":
                    previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                    break
            previous_exon_number -= 1
        while True:
            if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                next_exon_coordinates = coordinate_file_lines[next_exon_number]
                break
            else:
                proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[next_exon_number]}\nProceed?")
                if proceed_prompt.lower()[0] == "y":
                    next_exon_coordinates = coordinate_file_lines[next_exon_number]
                    break
            next_exon_number += 1

        
        print(previous_exon_coordinates)
        print(next_exon_coordinates)
        break
        # print(coordinate_file_lines[i])



Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length

Pararge_aegeria,NC_053208.1,16934367,16934455,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_1,1,29,29

Pararge_aegeria,NC_053208.1,16926960,16927135,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_3,1,57,58



In [24]:
previous_exon_coordinates
next_exon_coordinates
current_exon_coordinates
query_species = "5.Bicyclus_anynana"
query_transcript = "XM_024088150.2"
query_exon = current_exon_coordinates[6].split("query")[-1][1:]
error_exon = query_exon
# print(query_exon)
# assert False
query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/2.Blast/0.Query"
annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species"

genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
for file in list_of_files_in_genome_folder:
    if file.endswith("_genomic.fna"):
        genome_file = file

print(genome_file)

upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
print(upstream_exon_line)
complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]
print(complement,scaffold)

if complement == "0":
    gene_start = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    gene_end = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))

if complement == "1":
    gene_start = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    gene_end = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))

print(gene_start, gene_end)


print(upstream_exon_line)
print(downstream_exon_line)

# genome = SeqIO.parse(f"{genome_location}/{species}/{genome_file}", "fasta")
# for entries in genome:
#     # print(scaffold)
#     if scaffold == entries.id:
#         print(scaffold)
#         gene_sequence = entries.seq[gene_start-1:gene_end]
#         if complement == "1":
#             gene_sequence = gene_sequence.reverse_complement()
#         break

with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
    
    query_fasta_sequence = "".join(query_file.readlines())

list_of_annotated_genomes = os.listdir(annotated_genome_location)
annotated_species_name = ''
# print(annotated_species)
for annotated_species in list_of_annotated_genomes:
    if annotated_species.endswith(species):
        annotated_species_name = (annotated_species)
        break
if annotated_species_name == '':
    print(f"Error with annotated species name")
    assert False

list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
query_length = len(query_fasta_sequence.split("\n")[1])


list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/")
if error_exon not in list_of_exon_directories:
    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
elif error_exon in list_of_exon_directories:
    list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
    if "for_alignment" not in list_of_folders_1:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
    list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
    for file in list_of_files:
        os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment/{file}")

# for offset in range(3):
# #     print()
#     translated_sequence = str(gene_sequence[offset:].translate()).split("*")
#     for i in range(len(translated_sequence)):
#         if len(translated_sequence[i])> 0.8*query_length:
#             sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
#             with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:

#                 output = f"{query_fasta_sequence}\n\n{sequence_set}"
#                 out_file.write(output)

GCF_905163445.1_ilParAegt1.1_genomic.fna
['Pararge_aegeria', 'NC_053208.1', '16934367', '16934455', '1', 'N', '5.Bicyclus_anynana_XM_024088150.2_query_Exon_1', '1', '29', '29\n']
1 NC_053208.1
16927135 16934367
['Pararge_aegeria', 'NC_053208.1', '16934367', '16934455', '1', 'N', '5.Bicyclus_anynana_XM_024088150.2_query_Exon_1', '1', '29', '29\n']
['Pararge_aegeria', 'NC_053208.1', '16926960', '16927135', '1', 'N', '5.Bicyclus_anynana_XM_024088150.2_query_Exon_3', '1', '57', '58\n']


In [10]:
list_of_files_to_run_mafft_on = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}'
print(location)
# print(list_of_files_to_run_mafft_on)
output = ''
for file in list_of_files_to_run_mafft_on:
    if file.endswith(".fa"):
        command = f'"/home/saurav/miniconda3/envs/ncbi_datasets/bin/mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
        print(command)
        os.system(f'{command}')
        # break
# print(output)

/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/Exon_2
"/home/saurav/miniconda3/envs/ncbi_datasets/bin/mafft" --localpair --maxiterate 16 --reorder --distout "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/Exon_2/Exon_2_translated_genomic_sequence_152_frame2.fa" > "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/Exon_2/alignment_Exon_2_translated_genomic_sequence_152_frame2.fa.txt"


outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.520
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 2
done.

Progressive alignment ... 
STEP     1 /1 
done.
tbfast (aa) Version 7.520
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

    0 / 2
Segment   1/  1    1-  71
done 001-001-1  identical.   
dvtditr (aa) Version 7.520
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinement me

In [23]:
mafft_run_folder = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}"
list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
score_output = ''
score = 99
min_score_sequence = ''
alignmnet_file = ''
for file in list_of_files_in_mafft_run_folder:
    if file.endswith(".fa.hat2"):
        with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
            dist_matrix_list = dist_matrix_file.readlines()
        
        sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
        distance_score = float(dist_matrix_list[-1].rstrip())
        score_output += f"{sequence_name} =  {distance_score}\n"
        if distance_score < score:
            score = distance_score
            min_score_sequence = sequence_name
            alignment_file = f'alignment_{file.replace(".hat2",".txt")}'

alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
# print (records.id)
fasta_start_position = 0
fasta_end_position = 0
start_switch = 0
end_switch = 0
for records in alignment_file:
    
    if start_switch == 1 and end_switch == 1:
        print(f"{records.id}\n{records.seq[fasta_start_position:fasta_end_position]}")
    if error_exon in records.id:
        for current_position in range(len(records.seq)):
            # print(records.seq[current_position])
            if "-" not in records.seq[current_position] and start_switch == 0:
                fasta_start_position = current_position
                start_switch = 1
            if end_switch == 1 and "-" not in records.seq[current_position]:
                end_switch = 0
                
            if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                fasta_end_position = current_position
                end_switch = 1

        print(fasta_start_position,fasta_end_position)
    
            
                
        print (records.id)
    # print(alignment_file_open.readlines())
#         print()    
#         print(dist_matrix_list[-1].rstrip())

print(score_output)
print(f"min = {min_score_sequence}, {score}" )

4 31
5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_2_query_Exon_2
set152_frame2
HSSKSTHSESNSSGSSGYGGKPSSSGY
set152_frame2 =  0.443

min = set152_frame2, 0.443


In [18]:



print(query_fasta_sequence)
# assert False

    

print(query_length)
print((gene_sequence))
sequence_set = ''

list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/")
if error_exon not in list_of_exon_directories:
    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")

Exon_2
>5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_2_query_Exon_2
HSSKSTHSSGSSGYGGKQTTSGY
33.Pararge_aegeria
23
GGTAAGATTTTAGCATTAGATAAGAATTAAGCTAATGAGTCATCGCCAAGTCATCACAACGAATGCGTCGTGCTTCATCACTGCGAAAAATTTTTGGCCGTCATTTTGACGTTTCACTTCTgtggaaaaaagaaaaaaaaaaggtcgacttttaactacttaaataaaggTCAATCCACAAAGGACGGAACTACACTAGCGGAACTAGATTGTTTGGCACAACACTGTGTCAATTTACCACTACACCGATTATATGGCACTCACTTTTAAAACTTGCGTACAAGCGAAACTTTTTATAAGTGGTTTTCTTCGCTGTCTTCACTTATGGCTATAAAACGGATTGATTACCAAATTCAAAGTCGCTCAGGGATCTATGAAGAGCTTAGAAGCGCTTATAGCTTAATGGTTGTGACATCGGCTTCACATGAGGGGGCACCGAGTTCGAaaccacctctaacttttcgtagttattCTGAGAGAAGGGCACGGGTGCACCCTTTACCCTATGGGCTGATAAAGGTAAGCTATTGACAGCTGACGgataaaattcgaaattaaaaGATCTGCAGGAGAACTCAAACAAGCTTAAATGGCAATGGGCACTGAGCAGGCCATGCTTATTATAGGTTTGAAGGCAGTTTAAGCGAAAGTCCTACTGGTTAGAACGTGGATAGTAGGCAATGCGAGTTGGACGAGCGATCGAGCGAGTTGGTTCGTTGGTTTGGAGTCCGTTCGTATCGTGTAATTATGGTGGCGCTTCCTAGGAAAGCTCTAAATAGAGGATTTAGCAGGAAACGTGCTCAGGCTCATGATAGATCATGATAGCCTGCCGACCGAGGATAGATAGAAATAGGAAATTCATTTTGCGTTATATTCATGCG

In [34]:
os.system(f'ls -l "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}"')

total 66
-rwxrwxrwx 0 saurav saurav 133 Feb 15 14:52 Exon_2_translated_genomic_sequence_101_frame2.fa
-rwxrwxrwx 0 saurav saurav 134 Feb 15 14:52 Exon_2_translated_genomic_sequence_102_frame0.fa
-rwxrwxrwx 0 saurav saurav 128 Feb 15 14:52 Exon_2_translated_genomic_sequence_102_frame1.fa
-rwxrwxrwx 0 saurav saurav 132 Feb 15 14:52 Exon_2_translated_genomic_sequence_104_frame2.fa
-rwxrwxrwx 0 saurav saurav 132 Feb 15 14:52 Exon_2_translated_genomic_sequence_106_frame0.fa
-rwxrwxrwx 0 saurav saurav 141 Feb 15 14:52 Exon_2_translated_genomic_sequence_106_frame1.fa
-rwxrwxrwx 0 saurav saurav 159 Feb 15 14:52 Exon_2_translated_genomic_sequence_107_frame1.fa
-rwxrwxrwx 0 saurav saurav 131 Feb 15 14:52 Exon_2_translated_genomic_sequence_107_frame2.fa
-rwxrwxrwx 0 saurav saurav 133 Feb 15 14:52 Exon_2_translated_genomic_sequence_108_frame2.fa
-rwxrwxrwx 0 saurav saurav 139 Feb 15 14:52 Exon_2_translated_genomic_sequence_109_frame0.fa
-rwxrwxrwx 0 saurav saurav 127 Feb 15 14:52 Exon_2_translated

0

nomic_sequence_156_frame1.fa
-rwxrwxrwx 0 saurav saurav 129 Feb 15 14:52 Exon_2_translated_genomic_sequence_15_frame2.fa
-rwxrwxrwx 0 saurav saurav 133 Feb 15 14:52 Exon_2_translated_genomic_sequence_16_frame2.fa
-rwxrwxrwx 0 saurav saurav 140 Feb 15 14:52 Exon_2_translated_genomic_sequence_17_frame1.fa
-rwxrwxrwx 0 saurav saurav 130 Feb 15 14:52 Exon_2_translated_genomic_sequence_19_frame1.fa
-rwxrwxrwx 0 saurav saurav 135 Feb 15 14:52 Exon_2_translated_genomic_sequence_20_frame2.fa
-rwxrwxrwx 0 saurav saurav 133 Feb 15 14:52 Exon_2_translated_genomic_sequence_21_frame0.fa
-rwxrwxrwx 0 saurav saurav 145 Feb 15 14:52 Exon_2_translated_genomic_sequence_22_frame1.fa
-rwxrwxrwx 0 saurav saurav 127 Feb 15 14:52 Exon_2_translated_genomic_sequence_24_frame1.fa
-rwxrwxrwx 0 saurav saurav 170 Feb 15 14:52 Exon_2_translated_genomic_sequence_25_frame2.fa
-rwxrwxrwx 0 saurav saurav 131 Feb 15 14:52 Exon_2_translated_genomic_sequence_26_frame0.fa
-rwxrwxrwx 0 saurav saurav 206 Feb 15 14:52 Exon_2_